In [1]:
import pandas as pd

In [2]:
BASE_PATH = '/home/baldasseroniluisesteban/buckets/b1/datasets/'

df = pd.read_csv(BASE_PATH + 'competencia_02_sin_nulos.csv.gz')
df.head()

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,865361379,202101,1,0,0,34,134,445.55,5462.66,385.70,...,4.0,0.0,-779.25,0.0,4074.0,779.25,2.0,0.0,35.19,CONTINUA
1,865361379,202102,1,0,0,34,135,525.97,5734.45,380.13,...,-3.0,0.0,-788.60,0.0,4102.0,861.70,2.0,0.0,35.19,CONTINUA
2,865361379,202103,1,0,0,34,136,2075.33,7553.68,1994.59,...,1.0,0.0,-872.05,0.0,4133.0,988.35,2.0,0.0,46.92,CONTINUA
3,865361379,202104,1,0,0,34,137,-1077.29,6202.98,-1159.95,...,2.0,0.0,-1000.21,0.0,4163.0,988.35,2.0,0.0,46.92,CONTINUA
4,865361379,202105,1,0,0,34,138,541.40,5178.31,475.75,...,5.0,0.0,-1000.21,0.0,4194.0,988.35,2.0,0.0,46.92,CONTINUA


In [ ]:
from scipy.stats import f_oneway, kendalltau

# Calcular correlaciones ANOVA (F-value)
correlaciones = {}
for columna in df.columns:
    # Dividir los datos por la categoría de 'clas_ternaria'
    grupos = [df[df['clase_ternaria'] == clase][columna] for clase in df['clase_ternaria'].unique()]
    
    # Calcular F-statistic (ANOVA)
    f_stat, p_value = f_oneway(*grupos)
    correlaciones[columna] = f_stat

# Crear un DataFrame con los resultados
df_correlaciones = pd.DataFrame.from_dict(correlaciones, orient='index', columns=['F-value'])
df_correlaciones = df_correlaciones.sort_values(by='F-value', ascending=False)

# Mostrar las columnas más correlacionadas
print(df_correlaciones.head())